<a href="https://colab.research.google.com/github/HIsu1231/Catholic_University_AI_Senior/blob/main/AI%EC%84%A0%EB%B0%B0_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. 드라이브 연동

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pandas as pd
import seaborn as sb
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text

1. 출신고교, 출신고교지역 전처리

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/AI선배/정보없음_0508_v3.csv', index_col=0) #csv파일 불러오기

In [ ]:
df

,년도,학기,생년,성별,학부/과코드,제1전공코드,제2전공코드,학년,이수학기,입학년도,...,교환학생(기간),전공성적평균,교양성적평균,기타성적평균,공통_영어평균,공통_종교평균,공통_팀플평균,공통_봉사평균,공통_1학년과목평균,label
학생구분번호,,,,,,,,,,,,,,,,,,,,,
M000000002,0,편입학기,1974,1,M220000014,M240000039,M240000004,4,7,1998,...,0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0
M000000002,2000,2학기,1974,1,M220000014,M240000039,M240000004,4,7,1998,...,0,3.125000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0
M000000002,1998,1학기,1974,1,M220000014,M240000039,M240000004,4,7,1998,...,0,2.500000,3.350000,0.0,0.0,0.0,0.0,0.0,0.0,0
M000000002,1999,1학기,1974,1,M220000014,M240000039,M240000004,4,7,1998,...,0,3.300000,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0
M000000002,2000,1학기,1974,1,M220000014,M240000039,M240000004,4,7,1998,...,0,2.100000,2.750000,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M000064917,2019,2학기,1999,1,M220000330,M230000358,M220000416,2,2,2019,...,0,4.000000,3.066667,0.0,3.5,0.0,2.5,0.0,3.5,0
M000059584,2017,1학기,1999,1,M220000328,M230000350,M240000004,2,2,2017,...,0,1.000000,1.250000,0.0,0.0,0.0,1.5,0.0,1.5,0
M000059584,2018,1학기,1999,1,M220000328,M230000350,M240000004,2,2,2017,...,0,0.000000,0.533333,0.0,0.0,0.0,2.5,0.0,0.0,0


In [ ]:
df2 = df[['출신고교','출신고교지역']]
high = df2.to_dict()
print(df2)
df2['출신고교'].value_counts()
school_place = df2.set_index('출신고교').T.to_dict('list')

                출신고교 출신고교지역
학생구분번호                     
M000000002       광신고  서울특별시
M000000002       광신고  서울특별시
M000000002       광신고  서울특별시
M000000002       광신고  서울특별시
M000000002       광신고  서울특별시
...              ...    ...
M000064917   광저우한국학교    NaN
M000059584   제물포고등학교  인천광역시
M000059584   제물포고등학교  인천광역시
M000068327  공주여자고등학교   충청남도
M000068327  공주여자고등학교   충청남도

[360188 rows x 2 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  """


In [ ]:
#지역별로 가장 많이 나온 학교 찾기
school_region = df.groupby('출신고교지역')['출신고교'].agg(**{'most_common_value':lambda x:x.mode()}).reset_index()
print(school_region.T.to_dict('list'))
print(type(school_region))

{0: ['강원도', '강릉여자고등학교'], 1: ['경기도', '소명여자고등학교'], 2: ['경상남도', '해성고등학교'], 3: ['경상북도', '포항제철고등학교'], 4: ['광주광역시', '광주인성고등학교'], 5: ['대구광역시', '명신여자고등학교'], 6: ['대전광역시', '대덕고등학교'], 7: ['부산광역시', '동래여자고등학교'], 8: ['서울특별시', '진명여자고등학교'], 9: ['세종특별시', array(['THE HIGHSCHOOL ATTACHED TO HARBIN INSTITUTE OF TECHNOLOGY',
       '北大??肇??育?院'], dtype=object)], 10: ['울산광역시', '울산여자고등학교'], 11: ['인천', '교동종합고등학교'], 12: ['인천광역시', '명신여자고등학교'], 13: ['전라남도', '광양제철고등학교'], 14: ['전라북도', '상산고등학교'], 15: ['정보없음', '정보없음'], 16: ['제주도', '신성여자고등학교'], 17: ['충청남도', '복자여자고등학교'], 18: ['충청북도', '제천고등학교'], 19: ['해외', '외국고']}
<class 'pandas.core.frame.DataFrame'>


In [ ]:
df2 = df[['출신고교','출신고교지역']]
pd.set_option('mode.chained_assignment',  None) #경고 메시지 없애기
df2['출신고교지역'] = df2['출신고교지역'].astype(str)

In [ ]:
#출신고교 지역도 
for i in range(len(df2)):
  if df2['출신고교지역'][i] == 'nan':
    df2['출신고교지역'][i] = '정보없음'
print(df2)

                출신고교 출신고교지역
학생구분번호                     
M000000002       광신고  서울특별시
M000000002       광신고  서울특별시
M000000002       광신고  서울특별시
M000000002       광신고  서울특별시
M000000002       광신고  서울특별시
...              ...    ...
M000064917   광저우한국학교   정보없음
M000059584   제물포고등학교  인천광역시
M000059584   제물포고등학교  인천광역시
M000068327  공주여자고등학교   충청남도
M000068327  공주여자고등학교   충청남도

[360188 rows x 2 columns]


In [ ]:
df2['출신고교'] = df2['출신고교'].astype(str) #출신고교 지역이 있으면, 해당 지역에서 최빈값인 출신고교로 값 바꾸기
for i in range(len(df2)):
  if df2['출신고교'][i] == 'nan':
    if df2['출신고교지역'][i] == '강원도':
      df2['출신고교'][i] = '강릉여자고등학교'
    elif df2['출신고교지역'][i] == '경기도':
      df2['출신고교'][i] = '소명여자고등학교'
    elif df2['출신고교지역'][i] == '경상남도':
      df2['출신고교'][i] = '해성고등학교'
    elif df2['출신고교지역'][i] == '경상북도':
      df2['출신고교'][i] = '포항제철고등학교'
    elif df2['출신고교지역'][i] == '광주광역시':
      df2['출신고교'][i] = '광주인성고등학교'
    elif df2['출신고교지역'][i] == '대구광역시':
      df2['출신고교'][i] = '명신여자고등학교'
    elif df2['출신고교지역'][i] == '대전광역시':
      df2['출신고교'][i] = '대덕고등학교'
    elif df2['출신고교지역'][i] == '부산광역시':
      df2['출신고교'][i] = '동래여자고등학교'
    elif df2['출신고교지역'][i] == '서울특별시':
      df2['출신고교'][i] = '진명여자고등학교'
    elif df2['출신고교지역'][i] == '세종특별시':
      df2['출신고교'][i] = 'THE HIGHSCHOOL ATTACHED TO HARBIN INSTITUTE OF TECHNOLOGY'
    elif df2['출신고교지역'][i] == '울산광역시':
      df2['출신고교'][i] = '울산여자고등학교'
    elif df2['출신고교지역'][i] == '인천':
      df2['출신고교'][i] = '교동종합고등학교'
    elif df2['출신고교지역'][i] == '인천광역시':
      df2['출신고교'][i] = '명신여자고등학교'
    elif df2['출신고교지역'][i] == '전라남도':
      df2['출신고교'][i] = '광양제철고등학교'
    elif df2['출신고교지역'][i] == '전라북도':
      df2['출신고교'][i] = '상산고등학교'
    elif df2['출신고교지역'][i] == '제주도':
      df2['출신고교'][i] = '신성여자고등학교'
    elif df2['출신고교지역'][i] == '충청남도':
      df2['출신고교'][i] = '복자여자고등학교'
    elif df2['출신고교지역'][i] == '충청북도':
      df2['출신고교'][i] = '제천고등학교'
    elif df2['출신고교지역'][i] == '해외':
      df2['출신고교'][i] = '외국고'

print(df2)

                출신고교 출신고교지역
학생구분번호                     
M000000002       광신고  서울특별시
M000000002       광신고  서울특별시
M000000002       광신고  서울특별시
M000000002       광신고  서울특별시
M000000002       광신고  서울특별시
...              ...    ...
M000064917   광저우한국학교   정보없음
M000059584   제물포고등학교  인천광역시
M000059584   제물포고등학교  인천광역시
M000068327  공주여자고등학교   충청남도
M000068327  공주여자고등학교   충청남도

[360188 rows x 2 columns]


In [ ]:
df2.to_csv("./df2_정보없음_0508.csv")

In [ ]:
#전처리 한 col 원래 파일에 넣어주기
df['출신고교_전처리'] = df2['출신고교']
df['출신고교지역_전처리'] = df2['출신고교지역']
df

,년도,학기,생년,성별,학부/과코드,제1전공코드,제2전공코드,학년,이수학기,입학년도,...,교양성적평균,기타성적평균,공통_영어평균,공통_종교평균,공통_팀플평균,공통_봉사평균,공통_1학년과목평균,label,출신고교_전처리,출신고교지역_전처리
학생구분번호,,,,,,,,,,,,,,,,,,,,,
M000000002,0,편입학기,1974,1,M220000014,M240000039,M240000004,4,7,1998,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,광신고,서울특별시
M000000002,2000,2학기,1974,1,M220000014,M240000039,M240000004,4,7,1998,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,광신고,서울특별시
M000000002,1998,1학기,1974,1,M220000014,M240000039,M240000004,4,7,1998,...,3.350000,0.0,0.0,0.0,0.0,0.0,0.0,0,광신고,서울특별시
M000000002,1999,1학기,1974,1,M220000014,M240000039,M240000004,4,7,1998,...,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,광신고,서울특별시
M000000002,2000,1학기,1974,1,M220000014,M240000039,M240000004,4,7,1998,...,2.750000,0.0,0.0,0.0,0.0,0.0,0.0,0,광신고,서울특별시
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M000064917,2019,2학기,1999,1,M220000330,M230000358,M220000416,2,2,2019,...,3.066667,0.0,3.5,0.0,2.5,0.0,3.5,0,광저우한국학교,정보없음
M000059584,2017,1학기,1999,1,M220000328,M230000350,M240000004,2,2,2017,...,1.250000,0.0,0.0,0.0,1.5,0.0,1.5,0,제물포고등학교,인천광역시
M000059584,2018,1학기,1999,1,M220000328,M230000350,M240000004,2,2,2017,...,0.533333,0.0,0.0,0.0,2.5,0.0,0.0,0,제물포고등학교,인천광역시


In [ ]:
df.to_csv("./출신고교_지역_전처리완료_0508.csv")

인덱스 값 주기

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/AI선배/출신고교_지역_전처리최종_0508.csv")

In [ ]:
df.columns #columns 출력

Index(['년도', '학기', '생년', '성별', '학부/과코드', '제1전공코드', '제2전공코드', '학년', '이수학기',
       '입학년도', '입학구분', '입학전형구분', '출신고교', '출신고교지역', '신청학점', '이수학점', '평균평점',
       '동아리', '장학금', '봉사활동시간', '학술활동', '교환학생(기간)', '전공성적평균', '교양성적평균',
       '기타성적평균', '공통_영어평균', '공통_종교평균', '공통_팀플평균', '공통_봉사평균', '공통_1학년과목평균',
       'label', '출신고교_전처리', '출신고교지역_전처리'],
      dtype='object')

In [ ]:
l = []
l = df['출신고교_전처리'].values
l = set(l) #set으로 하면 같은 값은 없어짐
l = list(l) #set을 list에 넣어줌
dic_school = {}
for i in range(len(l)): #for 문 사용해서 dic_school 출신고교 인덱스 달아주기
   a = str(l[i])
   dic_school[a] = i + 1

{'안양고등학교', '덕명정보여자고등학교', 'Gledwood House School College', 'Changsha Mingda Middle School', 'Instituto Flores', '태광고등학교', '검정고시(제주)', '건국대학교사범대학부', '김포대학', '예산고등학교', '신명여자고', '청주고등학교', '교동고등학교', '인천중앙여자상업고등학교', '풍문여자고', '담양고등학교', '현대고등학교', '독산고등학교', '고양외국어고등학교', '인천세무고등학교', '제주과학고등학교', '염광여자고등학교', '홍익대학교사범대학부속여자고등학교', '한양대학교', '명덕고', '화봉고등학교', '철광고등학교', '문태고등학교', '선영여자고등학교', '천안대학교', '고흥녹동고', '경남정보대학', '안양여자고', '長崎&#30476;立&#23550;馬高等&#23398;校', '서전고등학교', '순천제일대학', '문정여자고등학교', '용남고', '제주제일고등학교', 'International School of Manila', '삼육간호보건대학', '영주고등학교', '마산무학여자고등학교', 'HANGZHOU XIAOSHAN NO.11 SENIOR HIGH SCHOOL', '개포고등학교', '수내고등학교', '학점은행(대학과정)', '도봉정보산업고등학교', '서서울생활과학고등학교', 'Rio americano high school', '역곡고등학교', '혜원여자고', '강남대학교', '오현고', '성만여자상업고', '수도여자고', '서원고등학교', '양산제일고등학교', '성덕고등학교', '신현고등학교', '화성반월고등학교', '안강여자고등학교', '양명고', '부곡중앙고등학교', '원곡고등학교', '연세대학교(원주)', '양지고등학교', '하남경무직업학교', '대구가톨릭대학교', '분포고등학교', '서울여상고', '언남고', '연천종합고', '서울문화고등학교', '대구남산여자고등학교', '양영디지털고등학교', '우송고등학교', '옥련여자고등학교',

In [ ]:
dic = {}
for i in range(len(l)):
   a = str(l[i])
   dic[a] = i + 1
dic

{'안양고등학교': 1,
 '덕명정보여자고등학교': 2,
 'Gledwood House School College': 3,
 'Changsha Mingda Middle School': 4,
 'Instituto Flores': 5,
 '태광고등학교': 6,
 '검정고시(제주)': 7,
 '건국대학교사범대학부': 8,
 '김포대학': 9,
 '예산고등학교': 10,
 '신명여자고': 11,
 '청주고등학교': 12,
 '교동고등학교': 13,
 '인천중앙여자상업고등학교': 14,
 '풍문여자고': 15,
 '담양고등학교': 16,
 '현대고등학교': 17,
 '독산고등학교': 18,
 '고양외국어고등학교': 19,
 '인천세무고등학교': 20,
 '제주과학고등학교': 21,
 '염광여자고등학교': 22,
 '홍익대학교사범대학부속여자고등학교': 23,
 '한양대학교': 24,
 '명덕고': 25,
 '화봉고등학교': 26,
 '철광고등학교': 27,
 '문태고등학교': 28,
 '선영여자고등학교': 29,
 '천안대학교': 30,
 '고흥녹동고': 31,
 '경남정보대학': 32,
 '안양여자고': 33,
 '長崎&#30476;立&#23550;馬高等&#23398;校': 34,
 '서전고등학교': 35,
 '순천제일대학': 36,
 '문정여자고등학교': 37,
 '용남고': 38,
 '제주제일고등학교': 39,
 'International School of Manila': 40,
 '삼육간호보건대학': 41,
 '영주고등학교': 42,
 '마산무학여자고등학교': 43,
 'HANGZHOU XIAOSHAN NO.11 SENIOR HIGH SCHOOL': 44,
 '개포고등학교': 45,
 '수내고등학교': 46,
 '학점은행(대학과정)': 47,
 '도봉정보산업고등학교': 48,
 '서서울생활과학고등학교': 49,
 'Rio americano high school': 50,
 '역곡고등학교': 51,
 '혜원여자고': 52,
 '강남대학교': 53,
 '오현고': 5

In [ ]:
#이거랑 이거 위에랑 뭐가 다른지 확인하기
df['출신고교인덱스'] = '0'
for i in range(len(df)):
  a = df['출신고교_전처리'][i]
  df['출신고교인덱스'][i] = dic[a]

df['출신고교인덱스']



KeyboardInterrupt: ignored

3. 모델 돌려보기

In [ ]:
#데이터 인덱스 값으로 바꾸거나 모델에 넣을 수 있도록 바꾸기
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

df = pd.read_csv("/content/drive/MyDrive/AI선배/출신고교_지역_전처리최종_0508.csv") #csv 파일 불러오기

def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)


# 잘못 들어온 열 'Column1' 제거 -> 이거 뭐지?/ 민우한테 물어보기
#df = df.drop(['Column1'], axis='columns')
# 학생구분번호 제거
df = df.drop(['학생구분번호'], axis='columns')
# 국적 제거
df = df.drop(['국적'], axis='columns')
# 출신고교 제거
df = df.drop(['출신고교'], axis='columns')
# 출신고교지역 제거
df = df.drop(['출신고교지역'], axis='columns')

## 문자열 -> 수치화 ##
# 학부/과코드, 제1전공코드, 제2전공코드(M 빼기)
df['학부/과코드'] = df['학부/과코드'].str.replace('M','')
df['제1전공코드'] = df['제1전공코드'].str.replace('M','')
df['제2전공코드'] = df['제2전공코드'].str.replace('M','')
# 입학구분
change_value_dict = {'신입': 0,'편입': 1}
df = df.replace({'입학구분': change_value_dict})
# 학기 인덱스 주기
df['학기인덱스'] = '0'
m = []
m = df['학기'].values
m = set(m)
m = list(m)
dic_hak = {}
for k in range(len(m)):
   e = str(m[k])
   dic_hak[e] = k + 1
for p in range(len(df)):
  g = df['학기'][p]
  df['학기인덱스'][p] = dic_hak[g]
# 입학전형구분
country_from = list(set(df['입학전형구분'].to_list()))
num = 0
for key in country_from :
  change_value_dict = {key : num}
  df = df.replace({'입학전형구분': change_value_dict})
  num += 1
# 출신고교 인덱스 주기
df['출신고교인덱스'] = '0'
l = []
l = df['출신고교_전처리'].values
l = set(l)
l = list(l)
dic_school = {}
for i in range(len(l)):
   a = str(l[i])
   dic_school[a] = i + 1
for i in range(len(df)):
  c = df['출신고교_전처리'][i]
  df['출신고교인덱스'][i] = dic_school[c]
# 출신지역 인덱스 주기
df['출신고교지역인덱스'] = '0'
s = []
s = df['출신고교지역_전처리'].values
s = set(s)
s = list(s)
dic_region = {}
for i in range(len(s)):
   b = str(s[i])
   dic_region[b] = i + 1
for j in range(len(df)):
  d = df['출신고교지역_전처리'][j]
  df['출신고교지역인덱스'][j] = dic_region[d]
# df[col] = df[col].applymap({'value':'0', 'va'})

#df = df.astype('float32') -> 숫자형태로 왜 했어?
df.dtypes

년도              int64
학기             object
생년              int64
성별              int64
학부/과코드         object
제1전공코드         object
제2전공코드         object
학년              int64
이수학기            int64
입학년도            int64
입학구분            int64
입학전형구분          int64
신청학점            int64
이수학점            int64
평균평점          float64
동아리             int64
장학금             int64
봉사활동시간          int64
학술활동            int64
교환학생(기간)        int64
전공성적평균        float64
교양성적평균        float64
기타성적평균        float64
공통_영어평균       float64
공통_종교평균       float64
공통_팀플평균       float64
공통_봉사평균       float64
공통_1학년과목평균    float64
label           int64
출신고교_전처리       object
출신고교지역_전처리     object
학기인덱스          object
출신고교인덱스        object
출신고교지역인덱스      object
dtype: object

In [ ]:
#df = df.astype('float32')
#df.dtypes

ValueError: ignored

In [ ]:
df
df.to_csv("220508_전처리최종_ai선배_v2.0_노수민.csv")